In [ ]:
# include the necessary libraries for the code in the cell below
import numpy as np

In [ ]:
# -----------------------------------------------------------------------------------------------------------------
class BaseFunction:
    def __init__(self, f_low=2*np.pi, f_high=4*np.pi, t=[0], symmetry='even'):
        self.name       = 'BaseFunction'                    # name of the function type
        self.f_low      = f_low                             # lowest frequency
        self.f_high     = f_high                            # highest frequency
        self.t          = t                                 # array of time domain values to sample the function at
        self.signal     = self.create_function(self.t)      # the outbound signal
        self.symmetry   = symmetry                          # the symmetry of the function. 'odd', 'even', or 'none'

    # the base function. to be defined in subclasses.
    def create_function(self, t):
        return np.zeros(len(t))

    # a composite function creating by scattering the base function with amplitudes A at time shifts tshifts
    def create_composite(self, A):
        class_comp = np.convolve(self.signal, A, mode='same')
        return class_comp

    # simple call to normalize the function if so desired
    def normalize(self, signal):
        return signal/np.sqrt(np.mean(np.abs(signal)**2))

    def normalize_to_polyfit_region(self, signal, SampleRate=1000, NormWindow=1):
        WindowLen   = SampleRate*NormWindow
        t1          = int((len(self.t)-WindowLen)/2)
        t2          = int((len(self.t)+WindowLen)/2)
        signal      = signal/np.sqrt(np.mean(np.real(signal[t1:t2])**2))
        return signal

# -----------------------------------------------------------------------------------------------------------------
# creates a modified sum of sincs, bandlimited to be between f_low and f_high
class Sinc(BaseFunction):
    def __init__(self, f_low=2*np.pi, f_high=4*np.pi, t=[0], sinc_order=1):
        self.sinc_order = sinc_order        # controls the bandwidth of the sinc

        super().__init__(f_low=f_low, f_high=f_high, t=t)

        self.name       = 'Sinc'

    # modified sinc to the sinc_order order, with frequencies exclusively between w_high and w_low
    def create_function(self, t):
        new_t_high   = self.f_high*t/self.sinc_order
        new_t_low    = self.f_low*t/self.sinc_order
        ratio        = self.f_high/self.f_low
        # store func as a complex array
        func    = np.array((((ratio*(np.sin(new_t_high)/(new_t_high))**self.sinc_order) -
                             ((np.sin(new_t_low)/(new_t_low))**self.sinc_order)) +
                            1j*(((np.cos(new_t_high)/(new_t_high))**self.sinc_order) -
                             ((np.cos(new_t_low)/(new_t_low))**self.sinc_order))), dtype=complex)
        return func

# -----------------------------------------------------------------------------------------------------------------
class Gauss(BaseFunction):
    def __init__(self, t=[0]):
        super().__init__(t=t)

        self.name   = 'Gauss'

    # gaussian pulse
    def create_function(self, t):
        func = np.exp(-(2*np.pi*t)**2)
        return func

# -----------------------------------------------------------------------------------------------------------------
class SuperFlat(BaseFunction):
    def __init__(self, N=6, f_low=2*np.pi, f_high=4*np.pi, t=[0], fit_int=.5, fit_pts_cnt=14, symmetry='even'):
        self.fit_int        = fit_int                                   # interval in units of time to fit to
        self.fit_pts_cnt    = fit_pts_cnt                               # number of time steps to fit on
        self.fit_times      = np.linspace(-fit_int/2, fit_int/2, fit_pts_cnt)      # array of times to fit on
        self.N              = N                                         # order of polynomial fit (# coefs - 1)
        self.symmetry       = symmetry

        super().__init__(f_low=f_low, f_high=f_high, t=t, symmetry=self.symmetry)

        self.name           = 'SuperFlat'

    # a sine (for odd functions) or cosine (for even functions) at frequency defined by f_low + dw*n.
    # this function must be used elsewhere in a loop.
    def basis(self, times, n):
        dw = (self.f_high - self.f_low)/self.N
        g = np.zeros(len(times), dtype=complex)
        if self.symmetry == 'even':
            g = np.cos((self.f_low + dw*n)*times) + 1j*np.sin((self.f_low + dw*n)*times)
        elif self.symmetry == 'odd':
            g = np.sin((self.f_low + dw*n)*times) + 1j*np.cos((self.f_low + dw*n)*times)
        return g

    # superfunction with a constant slope region.
    # fits on fit_pts_cnt number of points over the interval self.fit_interval.
    def create_function(self, t):
        sol     = [71.40519127751465, -272.80135358204996, 468.9206733326654,
                   -460.9286176678669, 272.0073907158811, -91.12426245578455,
                   13.524067771868335]


        self.N  = len(sol) - 1
        func    = np.zeros(len(t), dtype=complex)
        for i in range(0, self.N+1):
            func += sol[i]*self.basis(t, i)
        return func

# -----------------------------------------------------------------------------------------------------------------
class SuperSlope(SuperFlat):
    def __init__(self, N=6, f_low=2*np.pi, f_high=4*np.pi, t=[0], fit_int=.5, fit_pts_cnt=14):
        super().__init__(N=N, f_low=f_low, f_high=f_high, t=t, fit_int=fit_int, fit_pts_cnt=fit_pts_cnt, symmetry='odd')

        self.name   = 'SuperSlope'

    # superfunction with a constant slope region.
    # fits on fit_pts_cnt number of points over the interval self.fit_interval.
    def create_function(self, t):
        sol     = [30.611785501887567, -114.01703715004284, 194.8800448951818,
                   -196.3760256855161, 125.16226638327635, -50.20347106103716,
                   11.688464509810723, -1.2148663179124775]

        self.N  = len(sol) - 1
        func    = np.zeros(len(t), dtype=complex)
        for i in range(0, self.N+1):
            func += sol[i]*self.basis(t, i)
        return func

# -----------------------------------------------------------------------------------------------------------------
class SuperSinc(BaseFunction):
    def __init__(self, f_low=2*np.pi, f_high=4*np.pi, t=[0], sinc_order=1, symmetry='even'):
        self.N          = 6
        self.symmetry   = symmetry
        self.sinc_order = sinc_order        # controls the bandwidth of the sinc

        super().__init__(f_low=f_low, f_high=f_high, t=t)

        self.name       = 'SuperSinc'

    # a sine (for odd functions) or cosine (for even functions) at frequency defined by f_low + dw*n.
    # this function must be used elsewhere in a loop.
    def basis(self, times, n):
        dw = (self.f_high - self.f_low)/(self.N-1)
        g = np.zeros(len(times), dtype=complex)
        if self.symmetry == 'even':
            g = np.cos((self.f_low + dw*n)*times) + 1j*np.sin((self.f_low + dw*n)*times)
        elif self.symmetry == 'odd':
            g = np.sin((self.f_low + dw*n)*times) + 1j*np.cos((self.f_low + dw*n)*times)
        return g

    # modified sinc to the sinc_order order, with frequencies exclusively between w_high and w_low
    def create_function(self, t):
        sol     = [15.408780770443117, -59.314865773150196, 107.55083439408003,
                   -114.80200472744407, 76.61598246463329, -30.55860760706851,
                   6.099786096397219]
        self.N  = len(sol)
        func    = np.zeros(len(t), dtype=complex)
        for i in range(0, self.N):
            func += sol[i]*self.basis(t, i)
        return func

# -----------------------------------------------------------------------------------------------------------------
# class for waves with manually defined frequencies
class SuperRandom(BaseFunction):
    def __init__(self, f_low=2*np.pi, f_high=4*np.pi, t=[0], sinc_order=1, symmetry='even', coefs=[0]):
        self.N          = 6
        self.symmetry   = symmetry
        self.sinc_order = sinc_order        # controls the bandwidth of the sinc
        self.coefs      = coefs

        super().__init__(f_low=f_low, f_high=f_high, t=t)

        self.name       = 'SuperSinc'

    # a sine (for odd functions) or cosine (for even functions) at frequency defined by f_low + dw*n.
    # this function must be used elsewhere in a loop.
    def basis(self, times, n):
        dw = (self.f_high - self.f_low)/(self.N-1)
        g = np.zeros(len(times), dtype=complex)
        if self.symmetry == 'even':
            g = np.cos((self.f_low + dw*n)*times) + 1j*np.sin((self.f_low + dw*n)*times)
        elif self.symmetry == 'odd':
            g = np.sin((self.f_low + dw*n)*times) + 1j*np.cos((self.f_low + dw*n)*times)
        return g

    # modified sinc to the sinc_order order, with frequencies exclusively between w_high and w_low
    def create_function(self, t):
        self.N  = len(self.coefs)
        func    = np.zeros(len(t), dtype=complex)
        for i in range(0, self.N):
            func += self.coefs[i]*self.basis(t, i)
        return func